In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.prompts import PromptTemplate
import json
import requests

api_key = os.getenv("OPENAI_API_KEY")
serper_api_key = os.getenv("SERP_API_KEY")
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "llmapps"

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(temperature=0)

In [10]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.pdf")
    return dl.load()

documents = find_md_files('../new_pdf')
len(documents)


1

In [11]:
from langchain.text_splitter import CharacterTextSplitter

pdf_text_splitter = CharacterTextSplitter(chunk_size=1500,chunk_overlap  = 200)
document_sections = pdf_text_splitter.split_documents(documents)
len(document_sections)


2

In [12]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

In [13]:

retriever = db.as_retriever(search_kwargs=dict(k=3))


In [14]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x174f2bb50>, search_type='similarity', search_kwargs={'k': 3})

In [15]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
query = "Tell me more about the graphic desingers"
qa.run(query)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Streaming LangChain activity to W&B at https://wandb.ai/zestt/llmapps/runs/9fc2x6dm
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.
Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


"I'm sorry, but based on the given context, there is no information about graphic designers."